In [1]:
# Load table into memory

from mylibrary.connections import cursor, conn, engine, Automapped_Base, session
import pandas as pd 
sql = """
select * from tt_h.anomalies_tfl_json
where tfl_message = 'ok'
"""
df = pd.read_sql(sql, conn)


/Users/robinlinacre/anaconda/lib/python3.6/site-packages/sqlalchemy/dialects/postgresql/base.py:2526: SAWarning: Did not recognize type 'geometry' of column 'geom'
  (attype, name))


In [2]:
# Parse the above 
import json

for r in df.iterrows():
    
    index = r[0]
    row = r[1]
    
    tfl_response = row["tfl_response"]
    j1 = json.loads(tfl_response)
    journeys = j1["journeys"]

    # Keep only the fastest journey
    d = 1000
    for j in journeys:
        if j['duration'] < d:
            d = j['duration']
            fastest_journey = j

    df.loc[index, "duration"] = fastest_journey["duration"]
    df.loc[index, "fastest_journey_arrival_icscode"] = fastest_journey["legs"][-1]["arrivalPoint"]["icsCode"]
    df.loc[index, "fastest_journey_json"] = json.dumps(fastest_journey )

In [3]:
df2 = df[df["station_name_arrive"] != "Stratford"]
df2 = df2.sort_values(["station_name_depart", "duration"]).drop_duplicates("station_name_depart")

In [4]:
df2 = df2[["icscode_depart", "station_name_depart", "icscode_arrive", "station_name_arrive", "duration", "fastest_journey_arrival_icscode", "fastest_journey_json"]]
df2

,icscode_depart,station_name_depart,icscode_arrive,station_name_arrive,duration,fastest_journey_arrival_icscode,fastest_journey_json
17,1000006,Amersham,1001221,Paddington,32.0,1001221,"{""$type"": ""Tfl.Api.Presentation.Entities.Journ..."
26,90000281,Ashford International,1001276,St.Pancras,38.0,1001276,"{""$type"": ""Tfl.Api.Presentation.Entities.Journ..."
29,90000325,Audley End,1000138,Liverpool Street,39.0,1000138,"{""$type"": ""Tfl.Api.Presentation.Entities.Journ..."
41,1001009,Aylesbury,1001221,Paddington,45.0,1001221,"{""$type"": ""Tfl.Api.Presentation.Entities.Journ..."
7,1019670,Aylesbury Vale Parkway,1001221,Paddington,50.0,1001221,"{""$type"": ""Tfl.Api.Presentation.Entities.Journ..."
10,1000476,Bayford,1001221,Paddington,33.0,1001221,"{""$type"": ""Tfl.Api.Presentation.Entities.Journ..."
72,1000576,Bishops Stortford,1000138,Liverpool Street,37.0,1000138,"{""$type"": ""Tfl.Api.Presentation.Entities.Journ..."
82,1001033,Brimsdown,1000138,Liverpool Street,20.0,1000138,"{""$type"": ""Tfl.Api.Presentation.Entities.Journ..."


In [14]:
# Get 
sql = """
select id from tt_h.parsed_rail_journeys
where natrail_journey_minutes is null
"""

problem_stations = pd.read_sql(sql, conn)
ids = [int(i) for i in problem_stations["id"]]


In [62]:

import parse_tfl_json
from process_journeys import  add_cycle_and_total_time

row = df2[df2["icscode_depart"] == "1000006"].iloc[0]

journey = json.loads(row["fastest_journey_json"])

journeys = add_cycle_and_total_time([journey], "E145HP")
journey = joureys[0]

legs = journey["legs"]

summary = parse_tfl_json.summarise_legs(legs)
num_changes = parse_tfl_json.num_changes(legs)
final_arrival = parse_tfl_json.final_arrival(legs)
travel_time_minutes = parse_tfl_json.get_total_travel_time(legs) 
lat_lng = parse_tfl_json.lat_lng_from_legs(legs)


In [18]:
# Open each id and add 
import parse_tfl_json
from process_journeys import  add_cycle_and_total_time

Journeys = Automapped_Base.classes.parsed_rail_journeys

id = ids[0]
journey = session.query(Journeys).filter(Journeys.id == id).one()

anomaly_row = df2[df2["icscode_depart"] == journey.icscode].iloc[0]

journey = json.loads(anomaly_row["fastest_journey_json"])

journeys = add_cycle_and_total_time([journey], "E145HP")
journey = journeys[0]

legs = journey["legs"]

summary = parse_tfl_json.summarise_legs(legs)
num_changes = parse_tfl_json.num_changes(legs)
final_arrival = parse_tfl_json.final_arrival(legs)
travel_time_minutes = parse_tfl_json.get_total_travel_time(legs) 
lat_lng = parse_tfl_json.lat_lng_from_legs(legs)

journey.arrive_lat = 
journey.arrive_lng = 
journey.cycle_minutes = 
journey.cycle_miles = 
journey.total_journeytime = 

In [19]:


journey = session.query(Journeys).filter(Journeys.id == id).one()
journey
# df.loc[index, "arrive_lat"] = lat_lng["arrive"]["lat"]
# df.loc[index, "arrive_lng"] = lat_lng["arrive"]["lng"]
# df.loc[index, "cycle_minutes"] = journey["cycle_minutes"]
# df.loc[index, "cycle_miles"] = journey["cycle_miles"]
# df.loc[index, "total_journeytime"] = journey["cycle_minutes"] + travel_time_minutes